# Heat Exposure Prediction (Explainable ML + Spatial Block CV)

This notebook trains an **explainable** ML model to predict **seasonal mean temperature** (`mean_t`) from **urban form + greenness + land-use proxies**, and evaluates performance with:

- **Random KFold** (optimistic baseline)
- **Spatial Block CV** (reduces spatial leakage)
- (Optional) **Leave-location-out** CV using `id` groups

Dataset file: `data_output.csv`


In [ ]:
# If running on Google Colab, uncomment:
# !pip -q install pandas numpy scikit-learn matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
DATA_PATH = "data_output.csv"  # <- change if needed
TARGET = "mean_t"
LAT_COL, LON_COL = "LAT", "LON"
ID_COL = "id"

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
display(df.head())
print("\nColumns:", list(df.columns))


In [ ]:
# --- Basic checks ---
assert TARGET in df.columns, f"Target column {TARGET} not found"
assert LAT_COL in df.columns and LON_COL in df.columns, "LAT/LON columns not found"

print("Years:", sorted(df['year'].unique()) if 'year' in df.columns else "NA")
print("Seasons:", sorted(df['season'].unique()) if 'season' in df.columns else "NA")
print("Unique locations (id):", df[ID_COL].nunique() if ID_COL in df.columns else "NA")

missing = df.isna().mean().sort_values(ascending=False)
display(missing.head(15))


In [ ]:
# --- Feature set ---
DROP_COLS = [TARGET, 'max_t','min_t','diff_t', LAT_COL, LON_COL]  # avoid leakage from other temperature vars + coordinates
if ID_COL in df.columns:
    DROP_COLS.append(ID_COL)

X = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
y = df[TARGET].to_numpy()

cat_cols = [c for c in X.columns if X[c].dtype == 'object']
num_cols = [c for c in X.columns if c not in cat_cols]

print("Num features:", len(num_cols))
print("Cat features:", cat_cols)


In [ ]:
from sklearn.model_selection import KFold, GroupKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

preprocess = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median'))]), num_cols),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                      ('onehot', OneHotEncoder(handle_unknown='ignore'))]), cat_cols),
])

def make_spatial_blocks(df, block_deg=0.02, lat_col=LAT_COL, lon_col=LON_COL):
    latmin, lonmin = df[lat_col].min(), df[lon_col].min()
    bid = (np.floor((df[lat_col]-latmin)/block_deg).astype(int).astype(str) + "_" +
           np.floor((df[lon_col]-lonmin)/block_deg).astype(int).astype(str))
    return bid

blocks = make_spatial_blocks(df, block_deg=0.02)
print("Spatial blocks:", blocks.nunique())

# quick visual: blocks on map
plt.figure()
plt.scatter(df[LON_COL], df[LAT_COL], c=blocks.astype('category').cat.codes, s=8)
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.title("Spatial blocks (for CV grouping)")
plt.show()


In [ ]:
def cv_predict_regression(df, X, y, cv, groups=None, seed=42):
    preds = np.empty_like(y, dtype=float)
    for tr, te in cv.split(X, y, groups):
        model = RandomForestRegressor(
            n_estimators=120, random_state=seed, n_jobs=1,
            max_features='sqrt', min_samples_leaf=2
        )
        pipe = Pipeline([('prep', preprocess), ('model', model)])
        pipe.fit(X.iloc[tr], y[tr])
        preds[te] = pipe.predict(X.iloc[te])
    rmse = mean_squared_error(y, preds, squared=False)
    mae = mean_absolute_error(y, preds)
    r2 = r2_score(y, preds)
    return rmse, mae, r2, preds

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
gkf = GroupKFold(n_splits=5)

rmse_k, mae_k, r2_k, pred_k = cv_predict_regression(df, X, y, kfold, None)
rmse_b, mae_b, r2_b, pred_b = cv_predict_regression(df, X, y, gkf, blocks)

print("Random KFold   RMSE:", rmse_k, "MAE:", mae_k, "R2:", r2_k)
print("Spatial Block  RMSE:", rmse_b, "MAE:", mae_b, "R2:", r2_b)


In [ ]:
# Optional: Leave-location-out CV (stricter, groups by id)
if ID_COL in df.columns:
    gid = df[ID_COL]
    rmse_id, mae_id, r2_id, pred_id = cv_predict_regression(df, X, y, gkf, gid)
    print("Leave-location-out RMSE:", rmse_id, "MAE:", mae_id, "R2:", r2_id)
else:
    rmse_id = mae_id = r2_id = None


In [ ]:
# --- Diagnostic plots (Block CV) ---
residual = y - pred_b

plt.figure()
plt.scatter(y, pred_b, s=10)
plt.xlabel("Observed"); plt.ylabel("Predicted")
plt.title("Observed vs Predicted (Spatial Block CV, out-of-fold)")
plt.show()

plt.figure()
plt.scatter(df[LON_COL], df[LAT_COL], c=residual, s=10)
plt.xlabel("Longitude"); plt.ylabel("Latitude")
plt.title("Residual map (Observed - Predicted), Spatial Block CV")
plt.colorbar(label="Residual (°C)")
plt.show()


In [ ]:
# --- Explainability: permutation importance (planning-relevant drivers) ---
from sklearn.inspection import permutation_importance

# Fit on all data for interpretability (not used for evaluation)
final_model = Pipeline([('prep', preprocess),
                        ('model', RandomForestRegressor(
                            n_estimators=200, random_state=42, n_jobs=1,
                            max_features='sqrt', min_samples_leaf=2
                        ))])
final_model.fit(X, y)

# run importance on a subset for speed
idx = np.random.RandomState(42).choice(len(X), size=min(600, len(X)), replace=False)
pi = permutation_importance(final_model, X.iloc[idx], y[idx], n_repeats=15,
                            random_state=42, n_jobs=1, scoring='neg_root_mean_squared_error')

imp = pd.DataFrame({'feature': X.columns,
                    'importance_mean': pi.importances_mean,
                    'importance_std': pi.importances_std}).sort_values('importance_mean', ascending=False)

display(imp.head(20))

plt.figure(figsize=(8,5))
top = imp.head(12).iloc[::-1]
plt.barh(top['feature'], top['importance_mean'])
plt.xlabel("Permutation importance (ΔRMSE)")
plt.title("Top drivers (higher = more predictive)")
plt.tight_layout()
plt.show()


In [ ]:
# --- Export summary for 1-page PDF / README ---
summary = {
    "dataset_rows": int(df.shape[0]),
    "dataset_cols": int(df.shape[1]),
    "unique_locations": int(df[ID_COL].nunique()) if ID_COL in df.columns else None,
    "target": TARGET,
    "random_kfold": {"rmse": float(rmse_k), "mae": float(mae_k), "r2": float(r2_k)},
    "spatial_block_cv": {"rmse": float(rmse_b), "mae": float(mae_b), "r2": float(r2_b)},
    "leave_location_out": None if ID_COL not in df.columns else {"rmse": float(rmse_id), "mae": float(mae_id), "r2": float(r2_id)},
    "top_features": imp.head(12).to_dict(orient='records')
}

import json, os
os.makedirs("outputs", exist_ok=True)
with open("outputs/summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("Saved outputs/summary.json")
